# Neuro4ML Week 8: Neuromorphic Computing

## Introduction 

Neuromorphic engineering is a field that aims to design and build artificial neural systems that mimic the architecture and principles of biological neural networks. Unlike traditional von Neumann computing architectures, neuromorphic chips:

1. Process information in a parallel, event-driven manner
2. Integrate memory and computation
3. Operate with extremely low power consumption

### Why Trade-off Power and Accuracy?

Traditional deep learning models running on GPUs or CPUs consume significant power (often hundreds of watts). In contrast, the human brain processes complex information while consuming only ~20 watts. Neuromorphic chips aim to bridge this efficiency gap by:

- Using spike-based computation
- Implementing local learning rules
- Exploiting sparse, event-driven processing

However, these benefits often come with reduced accuracy compared to traditional deep learning approaches. Understanding and optimizing this trade-off is crucial for deploying neural networks in power-constrained environments like mobile devices or IoT sensors.

## Exercise Overview

In this exercise, you will:
1. Implement a simple neuromorphic chip simulator
2. Train SNNs with different architectures
3. Analyze the power-accuracy trade-off
4. Explore how different parameters affect this trade-off

This will also serve as a solid introduction on how to effectively train SNNs using modern packages such as SNNTorch ! 

In [7]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install snntorch
    !https://github.com/GabrielBena/neuro4ml_week_8.git
    import sys
    sys.path.append('neuro4ml_week_8')
    

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

from chip import NeuromorphicChip
from models import SNNModel

# For automatic reloading of external modules
%load_ext autoreload
%autoreload 2

## Exercise 1: Mapping

To complete this first question you need to implement the functions necessary to map your network on the chip.

- Go to chip.py and implement the `calculate_memory_usage` and `map` methods
- Go to models.py and implement the `n_neurons` and `n_synapses` properties
- Run the following cell to check your implementation

This is what you should see:

![Expected simulation results](sim_results.png)


In [9]:
chip = NeuromorphicChip()

dims = (128, 100, 10)
n_timesteps = 100
seed = 42
snn = SNNModel(n_in=dims[0], n_hidden=dims[1], n_out=dims[-1], beta=0.95, seed=seed)

In [ ]:
# Generate random input (seed is fixed to 42 for reproducibility)
torch.manual_seed(seed)
input_data = torch.randn(1, n_timesteps, dims[0]) * 10  # 100 timesteps

# Map the network on the chip
chip.map(snn)
# Run the network
output, results = chip.run(input_data=input_data)

print("\nSimulation Results:")
print(f"Energy consumption: {results['total_energy_nJ']/1000:.2f} µJ")
print(f"Memory usage: {results['memory_usage_bytes']/1024:.2f} KB")
print(f"Total neuron updates: {results['neuron_updates']}")
print(f"Total synapse events: {results['synapse_events']}")
print(f"Average spike rate: {results['spike_rate']:.3f}")
print(f"Total spikes: {results['total_spikes']}")

## Exercise 2: Proper Training

In this exercise you will train a SNN on the Randman dataset.

- Go to training.py and complete the `SNNTrainer` class, in particular the `calculate_accuracy` method
- Run the following cell to train your network
- Take a look at the training and testing metrics, especially the accuracy and energy consumption
- Start experimenting with different architectures and parameters to see how they affect the accuracy and energy consumption


In [29]:
from training import get_dataloaders, SNNTrainer

In [37]:
# Create dataloaders
train_loader, test_loader, dataset = get_dataloaders(
    batch_size=64,
)

In [ ]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)  # batch_size x timesteps x n_in. 1st and 2nd dims are swapped when passed to the model

In [53]:
snn_config = {
    "n_hidden": 128,
    "beta": 0.95,
    "seed": 42,
}

In [54]:
# Initialize model
snn = SNNModel(
    n_hidden=snn_config["n_hidden"],
    beta=snn_config["beta"],
    seed=snn_config["seed"],
)

In [ ]:
# Initialize trainer
trainer = SNNTrainer(snn, learning_rate=1e-3, lr_gamma=0.9, config=snn_config)
# Train the model
trainer.train(train_loader, test_loader, n_epochs=10)

### Plot the results
- We can plot the accuracy and energy consumption as a function of the epoch
- We see that the accuracy is improving but the energy consumption is also increasing
- This is a trade-off that we need to be aware of when training SNNs


In [ ]:
results = trainer.pd_results
fig, ax = plt.subplots()
sns.lineplot(
    data=results, x="epoch", y="accuracy", ax=ax, label="Accuracy", legend=False
)
ax2 = ax.twinx()
sns.lineplot(
    data=results,
    x="epoch",
    y="total_energy_nJ",
    ax=ax2,
    color="orange",
    label="Energy",
    legend=False,
)
ax.figure.legend()
ax.set_title(
    f"Accuracy and Energy, Final Trade-off Score: {trainer.pareto_tradeoff:.2f}"
)
plt.show()

## Exercise 3: Optimizing the trade-off

Now, you will explore how different parameters affect the accuracy and energy consumption of the SNN. This part is open-ended, here are some ideas:

- Experiment with network architectures (number of layers, number of neurons, etc.)
- Implement a temporal loss (time-to-first-spike), using SnnTorch. Be careful to change the `calculate_accuracy` method in `training.py`
- Implement a double exponential neuron model, using SnnTorch (snn.neurons.Synaptic)
- Regularize spiking activity. 
- Implement weight masks to reduce the number of synapses
- Use SnnTorch to make the time-constants heterogeneous and / or learnable, and maybe use less neurons

Ideally, after experimenting with these parameters, you should start to see a ruff trade-off between accuracy and energy ! Can we see some kind of Pareto front appearing ? 

The group that will get the best trade-off score will win the competition ! 